<img src="img/ENSAE_logo.jpg">

# *ELTDM Project*
# *Parallelised Community Detection with Spark MapReduce*  
**Mastère Spécialisé Data Science, 2018-19, "Éléments logiciels pour le traitement des données massives"** 
  
**Alban CHAMPEVILLE DE BOISJOLLY**   
**Yosuke IJIRI**  
  

# Abstract

This project implements a parallelised community detection algorithm proposed by [*"S. Moon, et al., Parallel community detection on large graphs with MapReduce and GraphChi, Data Knowl. Eng. (2015)"*.](http://dx.doi.org/10.1016/j.datak.2015.05.001) The model is based on Girvan-Newman Algorithm. We use Python, Spark and AWS as the environment.

# 1. Introduction

Analysing network datum has become a key issue in many areas to base suggestions, advices and strategies in many sectors and applications. Community detection aims at identifying clusters in a network, i.e. regroup vertices into groups of high density connected vertices i.a. to make friends recommendations or target marketing. The Girvan-Newman algorithm is one of the most used for this objective. It uses the concept of edge betweenness, which is a measure of the centrality and influence of an edge in a network, with the shortest-path method.

Given the size of networks and the method used to compute the edge betweenness, parallelization of the computation is unavoidable to realize the task efficiently. Our work is focused on the Shortest-Path Betweeness MapReduce Algorithm (SPB-MRA). Parallelizing computations will be realized at each of the 4 stages of the Map-Reduce algorithm.

Related article:  " Parallel community detection on large graphs with MapReduce and GraphChi" published in 2015 by Seunghyeon Moon, Jae-Gil Lee, Minseo Kang, Minsoo Choy and Jin-woo Lee.

# 2. Model

Community Detection, Modularity Method

# 3. Algorithm

Girvan-Newman Algorithm

<img src="img/algo.png" width="400" align="left">

# 4. Data

Stanford Large Network Dataset Collection (https://snap.stanford.edu/data/)

# 5. Environment

Python + Spark + AWS

# 6. Implementation

Here is the code:

In [3]:
import numpy as np
import pandas as pd

In [4]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [5]:
import os
os.getcwd()

'/Users/IjiriYosuke/Documents/GitHub/ELTDM-Project'

In [6]:
# Read the data
#data = np.loadtxt("./Data/ca-GrQc_small.txt", dtype='uint16')
data = np.loadtxt("./Data/test2.txt", dtype='uint16')
print(data)
print(type(data))
#data = pd.DataFrame(data)
#data.to_csv("./Data/test_with_index.txt", sep="\t", header=None, index=False)

[[1 2]
 [1 3]
 [2 3]
 [4 3]
 [3 5]
 [5 6]
 [5 7]
 [1 8]
 [8 3]
 [6 8]]
<class 'numpy.ndarray'>


In [7]:
nodeId = np.unique(data)
nrow = len(data)
print("The data contains")
print(len(nodeId), "nodes, and", nrow, "edges (including duplicates)")

The data contains
8 nodes, and 10 edges (including duplicates)


In [211]:
# Make a list of adjacency lists (first element is the node of origin)
adj_list = []
for n in nodeId:
    adj = []
    for i in range(nrow):
        if data[i, 0] == n:
            adj.append(data[i, 1])
        if data[i, 1] == n:
            adj.append(data[i, 0])
    adj = list(set(adj)) 
    adj.insert(0, n)
    adj_list.append(adj)
#print(adj_list)

In [212]:
# Make tuple for each node
class tuples():
    def __init__(self, targetId, sourceId, distance, status, weight, pathInfo, adjList) :
        self.targetId = targetId
        self.sourceId = sourceId
        self.distance = distance
        self.status = status
        self.weight = weight
        self.pathInfo = pathInfo
        self.adjList = adjList

network = []
for n in nodeId:
    v = tuples(targetId = n, sourceId = n, distance = 0, status = 'a', weight = 1, pathInfo = [], adjList = [])
    for i in range(len(adj_list)):
        if adj_list[i][0] == n:
            v.adjList = adj_list[i][1:len(adj_list[i])]
            break
    network.append([v.targetId, v.sourceId, v.distance, v.status, v.weight, v.pathInfo, v.adjList])
#for i in range(len(network)):
#    print(network[i])

# Parallelisation part

In [10]:
#sc.stop()
from copy import deepcopy  # For making a deep copy of tuple
from itertools import groupby
from pyspark import SparkContext, SparkConf
conf = (SparkConf()
        .setMaster("local[*]")
        .setAppName("Test Name")
        .set('spark.executor.memory', '14g')
        .set('spark.driver.memory', '14g'))
sc = SparkContext(conf = conf)
#SparkConf().set('spark.driver.memory', '14g')
#SparkConf().set('spark.executor.memory', '14g')
#SparkConf().set('spark.app.name', 'test')
#sc = SparkContext(conf=SparkConf())
sc

<SparkContext master=local[*] appName=Test Name>

In [11]:
sc._conf.getAll()

[('spark.executor.memory', '14g'),
 ('spark.app.name', 'Test Name'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.port', '52854'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1544779514864'),
 ('spark.driver.memory', '14g'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.host', '172.16.41.210')]

In [309]:
# Set a dataset and create key-value pairs
rdd = sc.parallelize(network)
rdd = rdd.map(lambda x: (x[0], x[1:]))
output = rdd.collect()

# < Stage 1 - Map >

In [13]:
def stage1_map(p):  
    result = []
    if p[1][2] == 'a':
        p[1][2] = 'i'
        p[1][1] += 1
        p[1][4].append(p[0])
        x = deepcopy(p)
        result.append(x)
        temp = p[1][5].copy()
        for i in range(len(temp)):
            k = temp[i]
            p[1][2] = 'a'
            p[1][5] = []
            y = deepcopy(p)
            result.append((k, y[1]))
        return result
    else:
        els = deepcopy(p)
        result.append(els)
        return result
'''
pos_map1 = rdd.flatMap(stage1_map)
pos_map1.collect()
'''
# Add artifitial records in order to test MapReduce functions
#for_testing = pos_map1.collect()
#add_for_testing = [(4, [4, 2, 'a', 1, [2], []]), 
#    (4, [2, 1, 'a', 1, [3], [1, 3]]),
#    (4, [2, 2, 'a', 1, [2], []]),
#    (4, [2, 5, 'a', 1, [2], []]),
#    (4, [2, 1, 'a', 1, [3], [1, 3]]),
#    (4, [2, 1, 'a', 1, [5], [5]]),
#    (4, [2, 1, 'a', 1, [2], [2]])]
#for_testing.extend(add_for_testing)
#
#for i in range(len(for_testing)):
#    print(for_testing[i])
#pos_map1_test = sc.parallelize(for_testing)

'\npos_map1 = rdd.flatMap(stage1_map)\npos_map1.collect()\n'

# < Stage 1 - Reduce >

This part consists of 6 steps:  
1) Add *sourceId* to key  
2) Aggregate the values that belong to the same key  
3) Take only values that have the shortest distance in the same key  
4) Replace *weight* of such values $\in minList$ with the size of $minList$  
5) Flatten values and list the tuples  
6) Fill in any empty adjList

In [14]:
#pos_map1.collect()
#pos_map1_test.collect()
'''
red1_twokeys = pos_map1.map(lambda x: ((x[0],x[1][0]), x[1][1:]))
'''
#red1_twokeys = pos_map1_test.map(lambda x: ((x[0],x[1][0]), x[1][1:]))
#red1_twokeys.collect()

'\nred1_twokeys = pos_map1.map(lambda x: ((x[0],x[1][0]), x[1][1:]))\n'

In [15]:
def join(x, y):
    # 4 conditional statements below to join values
    # with correct level of nesting
    if (type(x[0]) is list) and (type(y[0]) is not list):
        return x+[y]
    elif (type(x[0]) is not list) and (type(y[0]) is list):
        return [x]+y
    elif (type(x[0]) is not list) and (type(y[0]) is not list):
        return [x]+[y]
    else:
        return x+y
    isinstance(x[0], int)
'''
red1_grouped = red1_twokeys.reduceByKey(join)
#red1_twokeys.reduceByKey(join).collect()
'''

'\nred1_grouped = red1_twokeys.reduceByKey(join)\n#red1_twokeys.reduceByKey(join).collect()\n'

In [16]:
def stage1_reduce(x):
    if type(x[1][0]) is list:
        m = x[1][0][0]
        for s in range(len(x[1])):
            if x[1][s][0] < m:
                m = x[1][s][0]
        minList = []
        # Drop tuple(s) with non-shortest distance
        for s in range(len(x[1])):
            if x[1][s][0] == m:
                minList.append(x[1][s])
        y = (x[0], minList)
        # Update weight
        for s in range(len(minList)):
            y[1][s][2] = len(minList)
        return y
    else:
        return x
'''
red1_weighted = red1_grouped.map(stage1_reduce)
#red1_grouped.map(stage1_reduce).collect()
'''

'\nred1_weighted = red1_grouped.map(stage1_reduce)\n#red1_grouped.map(stage1_reduce).collect()\n'

In [17]:
def twokeys_to_onekey(x):
    k = x[0][0]
    v = [x[0][1]]
    v.extend(x[1])
    return k, v
'''
# From two-keys tuples to one-key tuples
record = red1_weighted.map(twokeys_to_onekey).collect()

# To flatten the grouped values (non-parallelized)
new = []
for i in range(len(record)):
    if type(record[i][1][1]) is list:
        record[i]
        key = record[i][0]
        srcId = [record[i][1][0]]
        others = record[i][1][1:]
        for j in range(len(others)):
            s = srcId.copy()
            o = others[j]
            s.extend(o)
            newrec = (key, s)
            new.append(newrec)
pos_red1 = list(filter(lambda item: type(item[1][1]) != list, record))  
pos_red1.extend(new)
for i in range(len(pos_red1)):
    print(pos_red1[i])
output = pos_red1.copy()
'''

'\n# From two-keys tuples to one-key tuples\nrecord = red1_weighted.map(twokeys_to_onekey).collect()\n\n# To flatten the grouped values (non-parallelized)\nnew = []\nfor i in range(len(record)):\n    if type(record[i][1][1]) is list:\n        record[i]\n        key = record[i][0]\n        srcId = [record[i][1][0]]\n        others = record[i][1][1:]\n        for j in range(len(others)):\n            s = srcId.copy()\n            o = others[j]\n            s.extend(o)\n            newrec = (key, s)\n            new.append(newrec)\npos_red1 = list(filter(lambda item: type(item[1][1]) != list, record))  \npos_red1.extend(new)\nfor i in range(len(pos_red1)):\n    print(pos_red1[i])\noutput = pos_red1.copy()\n'

In [310]:
import time
t_start = time.time() 
# Iteration until status becomes 'i' for all tuples
print("Data contains", len(nodeId), "nodes, and", nrow, "edges (including duplicates) \n")
act_counter = 1
ite = 1
while act_counter > 0:
    print("Iteration", ite, "started...")
    rdd = sc.parallelize(output)
# <stage1 map>
    print(" Now doing Map..")
    pos_map1 = rdd.flatMap(stage1_map)
#    test_only = pos_map1.collect()
#    print("After Map of iteration", ite, ":")
#    for i in range(len(test_only)):
#        print(test_only[i])
# <stage1 reduce>
    print(" Now doing Reduce..")
    red1_twokeys = pos_map1.map(lambda x: ((x[0],x[1][0]), x[1][1:]))
    red1_grouped = red1_twokeys.reduceByKey(join)
    red1_weighted = red1_grouped.map(stage1_reduce)
    print(" Now collecting Reduce result..")
    t1 = time.time()
    record = red1_weighted.map(twokeys_to_onekey).collect()
    t2 = time.time()
    print("  Collecting Reduce result took", round(t2-t1, 1), "seconds (", round((t2-t1)/60, 1), "mins ).")
    print(" Now flattening Reduce result..")
    t3 = time.time()
    # To flatten the grouped values (non-parallelized)
    new = []
    for i in range(len(record)):
        if type(record[i][1][1]) is list:
            record[i]
            key = record[i][0]
            srcId = [record[i][1][0]]
            others = record[i][1][1:]
            for j in range(len(others)):
                s = srcId.copy()
                o = others[j]
                s.extend(o)
                newrec = (key, s)
                new.append(newrec)
    pos_red1 = list(filter(lambda item: type(item[1][1]) != list, record))  # Save records with only one value
    pos_red1.extend(new)  # Add records which had multiple values
    output = pos_red1.copy()
    t4 = time.time()
    print("  Flattening took", round(t4-t3, 1), "seconds (", round((t4-t3)/60, 1), "mins ).")
    print(" Now finalising Reduce result by filling in adjList..")
    # Fill in adjList before going to next iteration
    t5 = time.time()
    for i in range(len(output)):
        if output[i][1][5] == []:
            for j in range(len(adj_list)):
                if adj_list[j][0] == output[i][0]:
                    output[i][1][5] = adj_list[j][1:len(adj_list[j])]
                    break
    t6 = time.time()
    print("  Filling in adjList took", round(t6-t5, 1), "seconds (", round((t6-t5)/60, 1), "mins ).")
    print(" Now iteration",ite ,"finished!")
    act_counter = 0
    for i in range(len(output)):
        if output[i][1][2] == 'a':
            act_counter += 1
    print("Iteration", ite, "finished. There are", act_counter, "/", len(output), "active tuples left. \n")
    ite += 1
stage1_output = output.copy()
print("All the iterations have finished. Stage1 was iterated for", ite-1, "times.")
print("Stage1 result is:")
for i in range(len(stage1_output)):
    print(stage1_output[i])
t_end = time.time()
elapsed_time = t_end-t_start
elapsed_time_min = elapsed_time/60
print("Total execution time was", round(elapsed_time, 1), "seconds (", round(elapsed_time_min, 1), "mins ).")

Data contains 8 nodes, and 10 edges (including duplicates) 

Iteration 1 started...
 Now doing Map..
 Now doing Reduce..
 Now collecting Reduce result..
 Now flattening Reduce result..
  Flattening took 0.0 seconds ( 0.0 mins ).
 Now finalising Reduce result by filling in adjList..
  Filling in adjList took 0.0 seconds ( 0.0 mins ).
 Now iteration 1 finished!
Iteration 1 finished. There are 20 / 28 active tuples left. 

Iteration 2 started...
 Now doing Map..
 Now doing Reduce..
 Now collecting Reduce result..
 Now flattening Reduce result..
  Flattening took 0.0 seconds ( 0.0 mins ).
 Now finalising Reduce result by filling in adjList..
  Filling in adjList took 0.0 seconds ( 0.0 mins ).
 Now iteration 2 finished!
Iteration 2 finished. There are 42 / 70 active tuples left. 

Iteration 3 started...
 Now doing Map..
 Now doing Reduce..
 Now collecting Reduce result..
 Now flattening Reduce result..
  Flattening took 0.0 seconds ( 0.0 mins ).
 Now finalising Reduce result by filling in a

In [311]:
len(stage1_output)

78

# < Stage2 - Map & Reduce>

In [312]:
pre_map2 = sc.parallelize(stage1_output)
# Stage2 Map
def stage2_map(x):
    edges_on_the_path = []
    for i in range(len(x[1][4])-1):
        k = (x[1][4][i], x[1][4][i+1])
        w = x[1][3]
        edges_on_the_path.append((k, 1/w))
    return edges_on_the_path
# Apply Map after filtering out tuples which has single node info and duplicated tuples (as we consider undirected graph network)
pos_map2 = pre_map2.filter(lambda x: x[0] > x[1][0]).flatMap(stage2_map)
#pos_map2.collect()
# Stage2 Reduce
from operator import add
pos_red2_directed = pos_map2.reduceByKey(add)
# We should ignore the order of node ID at each edge and run reduceByKey(add) again
pos_red2 = pos_red2_directed.map(lambda x: ((x[0][1], x[0][0]), x[1]) if x[0][0] > x[0][1] else x).reduceByKey(add)
pos_red2.collect()

[((6, 8), 4.666666666666666),
 ((2, 3), 5.166666666666666),
 ((3, 4), 7.0),
 ((1, 2), 1.8333333333333333),
 ((5, 6), 4.333333333333333),
 ((3, 8), 4.833333333333333),
 ((3, 5), 10.333333333333332),
 ((5, 7), 7.0),
 ((1, 3), 4.0),
 ((1, 8), 2.8333333333333335)]

# < Stage3 - Map & Reduce >

In [313]:
k_iter = 1  # We set k_iter = 1
# Stage3 Map & Reduce
distributed_cache = pos_red2.map(lambda x: (x[1], x[0])).sortByKey(False).map(lambda x: (x[1], x[0])).take(k_iter)
distributed_cache

[((3, 5), 10.333333333333332)]

In [315]:
pre_map2.filter(lambda x: x[0] > x[1][0]).collect()

[(7, [3, 3, 'i', 1, [3, 5, 7], [5]]),
 (8, [6, 2, 'i', 1, [6, 8], [1, 3, 6]]),
 (4, [2, 3, 'i', 1, [2, 3, 4], [3]]),
 (7, [4, 4, 'i', 1, [4, 3, 5, 7], [5]]),
 (4, [1, 3, 'i', 1, [1, 3, 4], [3]]),
 (8, [1, 2, 'i', 1, [1, 8], [1, 3, 6]]),
 (7, [5, 2, 'i', 1, [5, 7], [5]]),
 (3, [1, 2, 'i', 1, [1, 3], [1, 2, 4, 5, 8]]),
 (5, [3, 2, 'i', 1, [3, 5], [3, 6, 7]]),
 (7, [6, 3, 'i', 1, [6, 5, 7], [5]]),
 (4, [3, 2, 'i', 1, [3, 4], [3]]),
 (2, [1, 2, 'i', 1, [1, 2], [1, 3]]),
 (6, [1, 3, 'i', 1, [1, 8, 6], [8, 5]]),
 (3, [2, 2, 'i', 1, [2, 3], [1, 2, 4, 5, 8]]),
 (6, [5, 2, 'i', 1, [5, 6], [8, 5]]),
 (8, [3, 2, 'i', 1, [3, 8], [1, 3, 6]]),
 (5, [1, 3, 'i', 1, [1, 3, 5], [3, 6, 7]]),
 (6, [4, 4, 'i', 2, [4, 3, 8, 6], [8, 5]]),
 (6, [4, 4, 'i', 2, [4, 3, 5, 6], [8, 5]]),
 (8, [2, 3, 'i', 2, [2, 1, 8], [1, 3, 6]]),
 (8, [2, 3, 'i', 2, [2, 3, 8], [1, 3, 6]]),
 (5, [2, 3, 'i', 1, [2, 3, 5], [3, 6, 7]]),
 (8, [5, 3, 'i', 2, [5, 3, 8], [1, 3, 6]]),
 (8, [5, 3, 'i', 2, [5, 6, 8], [1, 3, 6]]),
 (6, [3, 3

# < Stage4 - Map >

In [324]:
def stage4_map(x):
    counter = 0
    output = []
    for i in range(len(distributed_cache)):
        if x[0] in distributed_cache[i][0]:
            counter += 1
            c = distributed_cache[i][0][not(distributed_cache[i][0].index(x[0]))]  # Take the corresponding node
            try:
                x[1][5].remove(c)  # Remove if the corresponding node exists in the adjList
            except ValueError:
                pass  # Pass otherwise
    if counter == 0:
        return [tuple(x)]
    else:
        new = (x[1][0], [x[1][0], 0, 'a', 1, [], []])
        return x, new
pos_map4 = pre_map2.flatMap(stage4_map)
pos_map4.collect()

[(7, [3, 3, 'i', 1, [3, 5, 7], [5]]),
 (5, [5, 1, 'i', 1, [5], [6, 7]]),
 (5, [5, 0, 'a', 1, [], []]),
 (7, [7, 1, 'i', 1, [7], [5]]),
 (1, [1, 1, 'i', 1, [1], [8, 2, 3]]),
 (3, [3, 1, 'i', 1, [3], [1, 2, 4, 8]]),
 (3, [3, 0, 'a', 1, [], []]),
 (8, [6, 2, 'i', 1, [6, 8], [1, 3, 6]]),
 (6, [8, 2, 'i', 1, [8, 6], [8, 5]]),
 (2, [4, 3, 'i', 1, [4, 3, 2], [1, 3]]),
 (1, [5, 3, 'i', 1, [5, 3, 1], [8, 2, 3]]),
 (4, [2, 3, 'i', 1, [2, 3, 4], [3]]),
 (4, [5, 3, 'i', 1, [5, 3, 4], [3]]),
 (7, [4, 4, 'i', 1, [4, 3, 5, 7], [5]]),
 (2, [3, 2, 'i', 1, [3, 2], [1, 3]]),
 (3, [4, 2, 'i', 1, [4, 3], [1, 2, 4, 8]]),
 (4, [4, 0, 'a', 1, [], []]),
 (1, [2, 2, 'i', 1, [2, 1], [8, 2, 3]]),
 (3, [8, 2, 'i', 1, [8, 3], [1, 2, 4, 8]]),
 (8, [8, 0, 'a', 1, [], []]),
 (5, [6, 2, 'i', 1, [6, 5], [6, 7]]),
 (6, [6, 0, 'a', 1, [], []]),
 (4, [1, 3, 'i', 1, [1, 3, 4], [3]]),
 (8, [1, 2, 'i', 1, [1, 8], [1, 3, 6]]),
 (7, [5, 2, 'i', 1, [5, 7], [5]]),
 (6, [6, 1, 'i', 1, [6], [8, 5]]),
 (4, [4, 1, 'i', 1, [4], [3]]),

# < Stage4 - Reduce >

In [334]:
def stage4_reduce(x, y):
    if x[0] == y[0]:
        #adj = y[5]
        return ('x', [x[0], 0, 'a', 1, [], [adj]])
    else:
        return x
pos_red4 = pos_map4.reduceByKey(stage4_reduce)
pos_red4.collect()

[(8, [6, 2, 'i', 1, [6, 8], [1, 3, 6]]),
 (4, [2, 3, 'i', 1, [2, 3, 4], [3]]),
 (5, ('x', [5, 0, 'a', 1, [], [[8, 1, 3, 6]]])),
 (1, [1, 1, 'i', 1, [1], [8, 2, 3]]),
 (6, [8, 2, 'i', 1, [8, 6], [8, 5]]),
 (2, [4, 3, 'i', 1, [4, 3, 2], [1, 3]]),
 (7, [3, 3, 'i', 1, [3, 5, 7], [5]]),
 (3, ('x', ['x', 0, 'a', 1, [], [[8, 1, 3, 6]]]))]

In [ ]:
#repeat until no edge to cut exists

# Visualisation Part

In [ ]:
# ------- UNDIRECTED
# Build a dataframe with your connections
# This time a pair can appear 2 times, in one side or in the other!
df = pd.DataFrame({ 'from':['D', 'A', 'B', 'C','A'], 'to':['A', 'D', 'A', 'E','C']})
df
# Build your graph. Note that we use the Graph function to create the graph!
G=nx.from_pandas_dataframe(df, 'from', 'to', create_using=nx.Graph() )

# Make the graphnx.draw(G, with_labels=True, node_size=1500, alpha=0.3, arrows=True)
plt.title("UN-Directed")

In [11]:
#Visualisation of the graph
#This is just for reference for now
import pandas
import networkx as nx
import matplotlib.pyplot as plt

def draw_graph(graph):

    # extract nodes from graph
    nodes = set([n1 for n1, n2 in graph] + [n2 for n1, n2 in graph])

    # create networkx graph
    G=nx.Graph()
    G.add_edge("Node1", "Node2", "Node3", "Node4","Node5", "Node6","Node7", "Node8")

    # add nodes
    for node in nodes:
        G.add_node(node)

    # add edges
    for edge in graph:
        G.add_edge(edge[0], edge[1])

    # draw graph
    pos = nx.shell_layout(G)
    nx.draw(G, pos, with_labels = True)

    # show graph
    plt.show()
    
draw_graph(data)


TypeError: add_edge() takes 3 positional arguments but 9 were given

ANNEX: Spark SQL - Tried Spark DataFrame but did not use it in the end

In [286]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import SQLContext
#sqlContext = SQLContext(sc)
spark = SparkSession.builder.appName('ELTDM').getOrCreate()
from pyspark.sql import functions as func
from pyspark.sql.functions import count, col
from pyspark.sql.window import Window
from pyspark.sql.functions import struct

from pyspark.sql.types import StructField
from pyspark.sql.types import StructType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import StringType
from pyspark.sql.types import ArrayType

In [316]:
fields = [
    StructField('targetId', IntegerType(), False),
    StructField('sourceId', IntegerType(), False),
    StructField('distance', IntegerType(), False),
    StructField('status', StringType(), False), 
    StructField('weight', IntegerType(), False),
    StructField('pathInfo', ArrayType(IntegerType(), True), False), 
    StructField('adjList', ArrayType(IntegerType(), True), False)]
schema = StructType(fields)

temp1 = pos_map1_test.map(lambda x: (int(x[0]),int(x[1][0]),int(x[1][1]),str(x[1][2]),int(x[1][3]),[int(s) for s in x[1][4]],[int(s) for s in x[1][5]])).collect()
temp2 = sc.parallelize(temp1)
df = spark.createDataFrame(temp2, schema)
df.show(100)

print(df.printSchema())
df.registerTempTable("Stage1_pre_reduce")

+--------+--------+--------+------+------+--------+------------+
|targetId|sourceId|distance|status|weight|pathInfo|     adjList|
+--------+--------+--------+------+------+--------+------------+
|       1|       1|       1|     i|     1|     [1]|[8, 2, 3, 4]|
|       8|       1|       1|     a|     1|     [1]|          []|
|       2|       1|       1|     a|     1|     [1]|          []|
|       3|       1|       1|     a|     1|     [1]|          []|
|       4|       1|       1|     a|     1|     [1]|          []|
|       2|       2|       1|     i|     1|     [2]|   [1, 4, 5]|
|       1|       2|       1|     a|     1|     [2]|          []|
|       4|       2|       1|     a|     1|     [2]|          []|
|       5|       2|       1|     a|     1|     [2]|          []|
|       3|       3|       1|     i|     1|     [3]|   [1, 4, 5]|
|       1|       3|       1|     a|     1|     [3]|          []|
|       4|       3|       1|     a|     1|     [3]|          []|
|       5|       3|      

In [381]:
q = """
SELECT
  targetId,
  sourceId,
  distance,
  status,
  weight,
  pathInfo,
  adjList
FROM
  Stage1_pre_reduce
ORDER BY targetId, sourceId
"""
df_ = sqlContext.sql(q)
df_.registerTempTable("Sorted")
df_.show()

+--------+--------+--------+------+------+--------+------------+
|targetId|sourceId|distance|status|weight|pathInfo|     adjList|
+--------+--------+--------+------+------+--------+------------+
|       1|       1|       1|     i|     1|     [1]|[8, 2, 3, 4]|
|       1|       2|       1|     a|     1|     [2]|          []|
|       1|       3|       1|     a|     1|     [3]|          []|
|       1|       4|       1|     a|     1|     [4]|          []|
|       1|       8|       1|     a|     1|     [8]|          []|
|       2|       1|       1|     a|     1|     [1]|          []|
|       2|       2|       1|     i|     1|     [2]|   [1, 4, 5]|
|       2|       4|       1|     a|     1|     [4]|          []|
|       2|       5|       1|     a|     1|     [5]|          []|
|       3|       1|       1|     a|     1|     [1]|          []|
|       3|       3|       1|     i|     1|     [3]|   [1, 4, 5]|
|       3|       4|       1|     a|     1|     [4]|          []|
|       3|       5|      

In [ ]:
q = """
SELECT
  targetId,
  sourceId,
  distance,
  MIN(distance) OVER (PARTITION BY sourceId ORDER BY targetId) as min_distance,
  status,
  weight,
  pathInfo,
  adjList
FROM
  Stage1_pre_reduce
"""
sqlContext.sql(q).show(100)

# 7. Result and discussion

bla bla bla

# 8.Efficiency

We can compare above result with that of non-parallelized algorithm.